In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import itertools
from collections import Counter
from scipy.stats import gaussian_kde
from moz_sql_parser import parse
sys.setrecursionlimit(3000)
import pickle

In [ ]:
saved_path = os.path.join(os.path.dirname(os.getcwd()), 'Saved Data\\')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 6]

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

In [ ]:
with open('req_tables.pickle', 'rb') as handle:
    dict_req_tables = pickle.load(handle)

In [ ]:
list_all_tables = []
for table in list(dict_req_tables.values()) :
    list_all_tables = list_all_tables + table 

In [ ]:
print('The number of unique tables is : ',len(set(list_all_tables)))

In [ ]:
'Frequency of tables on unique queries'
counts = Counter(list_all_tables)
counts = sorted(counts.items(), key = lambda x: x[1], reverse=True)
counts

In [ ]:
df_tables = pd.read_csv(saved_path + 'df_tables.csv', index_col=[0])
df_tables.head(5)

In [ ]:
top_tables = df_tables['table'].value_counts()[:50]
top_tables

In [ ]:
df = df_tables[df_tables['table'].isin(top_tables.index)]
g = sns.catplot(x="table", y="durationMS",jitter=False ,data=df,size = 12)
g.set_xticklabels(rotation=90)

In [ ]:
top_times = 250
rest_times = df_tables.shape[0] - top_times

data = df_tables.sort_values(by=['durationMS'],ascending = False)[:top_times]
data['table'].value_counts()

In [ ]:
rest_times

In [ ]:
g = sns.boxplot(x="table", y="durationMS", palette="husl", data=data)
g.set_xticklabels(g.get_xticklabels(),rotation=90)
g.set_yscale("log")

In [ ]:
_data = df_tables[df_tables['table'].isin(data['table'].value_counts().index)]
_data['table'].value_counts()

In [ ]:
g = sns.boxplot(x="table", y="durationMS", palette="husl", data=_data)
g.set_xticklabels(g.get_xticklabels(),rotation=90)
g.set_yscale("log")

In [ ]:
df_ratio = pd.DataFrame(data = pd.concat([data['table'].value_counts(), _data['table'].value_counts()], axis=1).values,
                        columns= ['top','rest'])
df_ratio = df_ratio.set_index(pd.concat([data['table'].value_counts(), _data['table'].value_counts()], axis=1).index)

df_ratio['rest'] = (df_ratio['rest'] - df_ratio['top']) / (rest_times - (df_ratio['rest'] - df_ratio['top']))
df_ratio['top'] = df_ratio['top'] / (top_times - df_ratio['top'])
df_ratio['ratio'] = df_ratio['top'] / df_ratio['rest']
df_ratio.sort_values(by='ratio', ascending=False)

In [ ]:
sns.FacetGrid(data, hue="table", palette="husl", size=10).map(sns.kdeplot, "durationMS").add_legend().set(yscale = 'log')

In [ ]:
df_tables_stat = pd.read_csv(saved_path + 'df_tables_stat.csv')

In [ ]:
df_tables_stat

In [ ]:
df_tables_stat.sort_values(by='moy', ascending=False)['table'][:20].values

In [ ]:
moy_data = df_tables[df_tables['table'].isin(df_tables_stat.sort_values(by='moy', ascending=False)['table'][:20].values)]
moy_data['table'].value_counts()